<h1><b>Exploring the Airbnb Market: Insights from Listings Data</b></h1>
<h3>Introduction</h3>
<p>The Airbnb market has revolutionized the travel and hospitality industry by offering diverse lodging options across the globe. Our dataset, containing 74,111 Airbnb listings from various cities, provides a snapshot of this dynamic market. This analysis aims to uncover pricing trends, popular property types, and host behaviors, ultimately offering insights for hosts and potential investors.</p>

<h3>Data Understanding and Preparation</h3> 
<p>The dataset includes a variety of features: 
<br>• Listings Information: Property type, room type, amenities, and accommodates. 
<br>• Pricing Details: Log-transformed price. 
<br>• Host Information: Host response rate, identity verification, and profile picture status. 
<br>• Geographical Data: Latitude, longitude, city, and neighborhood. 
<br>• Reviews: Number of reviews, review scores, and review dates.</p>

In [61]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
from scipy import stats

from geopy.distance import geodesic

In [62]:
df = pd.read_csv('Airbnb_Data.csv')
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [63]:
df.columns

Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds'],
      dtype='object')

In [64]:
# find out the shape of the dataset
print(f'The total rows and columns of the dataset is {df.shape[1]} and {df.shape[0]}')

The total rows and columns of the dataset is 29 and 74111


<h1>Data Wrangling</h1>

In [65]:
# check the column type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74111 entries, 0 to 74110
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      74111 non-null  int64  
 1   log_price               74111 non-null  float64
 2   property_type           74111 non-null  object 
 3   room_type               74111 non-null  object 
 4   amenities               74111 non-null  object 
 5   accommodates            74111 non-null  int64  
 6   bathrooms               73911 non-null  float64
 7   bed_type                74111 non-null  object 
 8   cancellation_policy     74111 non-null  object 
 9   cleaning_fee            74111 non-null  bool   
 10  city                    74111 non-null  object 
 11  description             74111 non-null  object 
 12  first_review            58247 non-null  object 
 13  host_has_profile_pic    73923 non-null  object 
 14  host_identity_verified  73923 non-null

In [66]:
# Calculate missing values for each column
missing_values = df.isnull().sum()

# Filter columns with missing values
missing_columns = missing_values[missing_values > 0]

# Display the columns with missing values and their count
missing_columns

bathrooms                   200
first_review              15864
host_has_profile_pic        188
host_identity_verified      188
host_response_rate        18299
host_since                  188
last_review               15827
neighbourhood              6872
review_scores_rating      16722
thumbnail_url              8216
zipcode                     966
bedrooms                     91
beds                        131
dtype: int64

#### Missing Data Overview
Our dataset reveals several columns with missing values, summarized as follows:

- **bathrooms**: 200 missing values
- **first_review**: 15,864 missing values
- **host_has_profile_pic**: 188 missing values
- **host_identity_verified**: 188 missing values
- **host_response_rate**: 18,299 missing values
- **host_since**: 188 missing values
- **last_review**: 15,827 missing values
- **neighbourhood**: 6,872 missing values
- **review_scores_rating**: 16,722 missing values
- **thumbnail_url**: 8,216 missing values
- **zipcode**: 966 missing values
- **bedrooms**: 91 missing values
- **beds**: 131 missing values


**Columns to Drop**:
- **first_review**: Due to a large number of missing values.
- **last_review**: Due to a large number of missing values.
- **host_response_rate**: Due to a large number of missing values.
- **thumbnail_url**: If not used for visual analysis
- **neighbourhood**: drop if non-critical.
- **zipcode**: drop if non-critical.

<h3>Drop the four columns: first_review, last_review, host_response_rate and thumbnail_url</h3>

In [67]:
# drop the four columns
df.drop(columns=["first_review","last_review","host_response_rate","thumbnail_url","neighbourhood","zipcode"],inplace=True)

**Columns to Keep (with Imputation)**:
- **bathrooms**: Impute with median.
- **host_has_profile_pic**: Impute with the most frequent value.
- **host_identity_verified**: Impute with the most frequent value.
- **host_since**: Impute with median or mode.
- **review_scores_rating**: Impute with median or a placeholder.
- **bedrooms**: Impute with median.
- **beds**: Impute with median.

<h3> Replace "bathrooms" columns missing values with median value</h3>

In [68]:
# fill the bathroom missing value with median value
median_bathroom = df["bathrooms"].astype("float").median(axis=0)
median_bathroom

1.0

In [69]:
# replace the missing bathrooms with median
df["bathrooms"].fillna(median_bathroom, inplace=True)
df['bathrooms'].isnull().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_19644\2495647314.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["bathrooms"].fillna(median_bathroom, inplace=True)


0

<h3> Replace "host_has_profile_pic" columns missing values with the most frequent value (mode)</h3>

In [70]:
# Calculate the most frequent value (mode) of the 'host_has_profile_pic' column
mode_host_pp = df["host_has_profile_pic"].mode()[0]

# Replace missing values in the 'host_has_profile_pic' column with the mode
df["host_has_profile_pic"].fillna(mode_host_pp, inplace=True)

# Print the DataFrame to verify the changes
df["host_has_profile_pic"].isnull().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_19644\2112208753.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["host_has_profile_pic"].fillna(mode_host_pp, inplace=True)


0

<h3> Replace "host_identity_verified" columns missing values with the most frequent value (mode)</h3>

In [71]:
# Calculate the most frequent value (mode) of the 'host_identity_verified' column
mode_verified_host = df['host_identity_verified'].mode()[0]

# Replace the missing value in the 'host_identity_verified' columns
df["host_identity_verified"].fillna(mode_verified_host, inplace=True)

# Print the dataframe to veryfy the changes
df["host_identity_verified"].isnull().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_19644\2944225036.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["host_identity_verified"].fillna(mode_verified_host, inplace=True)


0

<h3> Replace "host_since" columns missing values with the most frequent value (mode)</h3>

In [72]:
# Convert the "host_since" columns to datetime format 
df["host_since"] = pd.to_datetime(df["host_since"], errors="coerce")

In [73]:
# Calculate the median date of the "host_since" columns
median_host_since = df["host_since"].median()

# Replace the missing values in the "host_since" column with the median date
df['host_since'].fillna(median_host_since, inplace=True)

# Print the dataframe to verify changes
df["host_since"].isnull().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_19644\1023019723.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['host_since'].fillna(median_host_since, inplace=True)


0

<h3> Replace "review_scores_rating" columns missing values with placeholder</h3>

In [74]:
# Define the placeholder value
placeholder_value = -1

# Impute missing values in 'review_scores_rating' with the placeholder
df['review_scores_rating'].fillna(placeholder_value, inplace=True)

# Print the DataFrame to verify the changes
print(df['review_scores_rating'].isnull().sum())

0


C:\Users\User\AppData\Local\Temp\ipykernel_19644\2152306978.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['review_scores_rating'].fillna(placeholder_value, inplace=True)


<h3> Replace "bedrooms" columns missing values with the median</h3>

In [75]:
# Calculate the median of the 'bedrooms' column
median_bedrooms = df['bedrooms'].median()

# Replace the missing value in the 'bedrooms' columns
df["bedrooms"].fillna(median_bedrooms, inplace=True)

# Print the dataframe to veryfy the changes
df["bedrooms"].isnull().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_19644\1637036220.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["bedrooms"].fillna(median_bedrooms, inplace=True)


0

<h3> Replace "beds" columns missing values with the median</h3>

In [76]:
# Calculate the median of the 'beds' column
median_beds = df['beds'].median()

# Replace the missing value in the 'host_identity_verified' columns
df["beds"].fillna(median_beds, inplace=True)

# Print the dataframe to veryfy the changes
df["beds"].isnull().sum()

C:\Users\User\AppData\Local\Temp\ipykernel_19644\3403788569.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["beds"].fillna(median_beds, inplace=True)


0

<h3>Convert 'host_has_profile_pic','host_identity_verified','instant_bookable' columns to boolean</h3>
The instant_bookable, host_has_profile_pic, and host_identity_verified columns were stored as objects. Converting them to boolean would streamline any logical operations in the analysis.

In [85]:
# Columns with 't' and 'f' values
boolean_columns = ['host_has_profile_pic', 'host_identity_verified', 'instant_bookable']

# Covernt "t" to "True" and "f" to "False"
for col in boolean_columns:
    df[col] = df[col].map({'t': True, 'f': False})

# Ensure the columns are boolean type
for col in boolean_columns:
    df[col] = df[col].astype(bool)

# Verify the changes
print(df[boolean_columns].head())

   host_has_profile_pic  host_identity_verified  instant_bookable
0                  True                    True             False
1                  True                   False              True
2                  True                    True              True
3                  True                    True             False
4                  True                    True              True


In [87]:
df[['host_has_profile_pic', 'host_identity_verified', 'instant_bookable']].dtypes

host_has_profile_pic      bool
host_identity_verified    bool
instant_bookable          bool
dtype: object

<h3>Convert 'property_type', 'room_type', 'bed_type', 'cancellation_policy', 'city' columns to category</h3>
Certain columns, such as property_type, room_type, bed_type, cancellation_policy, city, and neighbourhood, contained categorical data. Converting these columns to the 'category' type would optimize memory usage and speed up analysis.

In [93]:
# Convert categorical columns to category type
categorical_columns = ['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'city']
for col in categorical_columns:
    df[col] = df[col].astype('category')

In [96]:
df[['property_type', 'room_type', 'bed_type', 'cancellation_policy', 'city']].dtypes

property_type          category
room_type              category
bed_type               category
cancellation_policy    category
city                   category
dtype: object

In [98]:
# Save the cleaned DataFrame to a CSV file
df.to_csv('cleaned_airbnb_dataset.csv', index=False)

print("Cleaned dataset saved to 'cleaned_airbnb_dataset.csv'")


Cleaned dataset saved to 'cleaned_airbnb_dataset.csv'
